In [56]:
%load_ext autoreload
%autoreload 2

import yaml
from IPython.core.display import HTML
from IPython.display import display
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2

from oml.models.siamese import ConcatSiamese
from oml.models.vit.vit import ViTExtractor
from oml.const import MOCK_DATASET_PATH
from oml.transforms.images.torchvision.transforms import get_normalisation_resize_hypvit
from oml.utils.images.images import imread_cv2, imread_pillow

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 330)

%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
extractor = ViTExtractor(arch="vits16", normalise_features=False, weights=None)
siamese = ConcatSiamese(extractor=extractor,
              mlp_hidden_dims=[192],
              weights="/nydl/logs/DeepFashion_InShop/2023-02-06_14-56-48_postprocessing/checkpoints/best.ckpt"
             )

Prefix <model.> was removed from the state dict.


In [23]:
df = pd.read_csv("/nydl/data/Stanford_Online_Products/df.csv")
df = df[df["split"] == "validation"]
df.reset_index(inplace=True, drop=True)


In [61]:
p1 = df["path"][90]
p2 = df["path"][98]

im1 = cv2.resize(imread_cv2(p1), (224, 224))
im2 = cv2.resize(imread_cv2(p2), (224, 224))
im = np.concatenate([im1, im2], axis=1)

tensor1 = get_normalisation_resize_hypvit(224, 224)(imread_pillow(p1)).unsqueeze(0)
tensor2 = get_normalisation_resize_hypvit(224, 224)(imread_pillow(p2)).unsqueeze(0)
out12 = siamese.predict(tensor1, tensor2)
out21 = siamese.predict(tensor2, tensor1)
print(out12.item(), out21.item())

attn = extractor.draw_attention(im1);
plt.imshow(attn)
plt.show()

attn = extractor.draw_attention(im2);
plt.imshow(attn)
plt.show()

attn = extractor.draw_attention(im);
plt.imshow(attn)
plt.show()

AttributeError: 'ConcatSiamese' object has no attribute 'predict'